<a href="https://colab.research.google.com/github/wendy705/guen_test_message/blob/main/Binder_Paraphraser_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 4.7 MB 16.1 MB/s 
     |████████████████████████████████| 596 kB 68.9 MB/s 
     |████████████████████████████████| 6.6 MB 63.6 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 1.3 MB 14.7 MB/s 


In [2]:
!pip install voila -q
!jupyter serverextension enable --sys-prefix voila

     |████████████████████████████████| 1.7 MB 9.3 MB/s 
     |████████████████████████████████| 54 kB 1.5 MB/s 
     |████████████████████████████████| 70 kB 5.2 MB/s 
     |████████████████████████████████| 563 kB 42.9 MB/s 
     |████████████████████████████████| 112 kB 45.3 MB/s 
     |████████████████████████████████| 344 kB 46.8 MB/s 
     |████████████████████████████████| 80 kB 8.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 423 kB 55.6 MB/s 
     |████████████████████████████████| 133 kB 69.1 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0, but you have tornado 6.2 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you 

In [3]:
from fastai.vision.widgets import *

In [9]:
from transformers import T5ForConditionalGeneration,T5Tokenizer
import torch

def t5_paraphrase_sentence(prompt, num_gen):
  
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  best_model_path = "/content/drive/MyDrive/t5_base_paraphrase_newalgo"
  model = T5ForConditionalGeneration.from_pretrained(best_model_path).to(device)
  tokenizer = T5Tokenizer.from_pretrained('google/mt5-base')

  text =  "paraphrase: " + prompt + " </s>"
  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      #min_length = text_len,
      temperature = 1.2,
      early_stopping=True,
      max_length = 256,
      num_return_sequences=num_gen
  )

  results = ""
  final_outputs = []
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != prompt.lower() and sent not in final_outputs:
          final_outputs.append(sent)
          #print(final_outputs)
          results.join(final_outputs)
  

  return final_outputs

In [5]:
text = input("Enter your sentence: ")
num_gen = input("Enter the number of paraphrases: ")

Enter your sentence: hello
Enter the number of paraphrases: 2


In [11]:
btn_run = widgets.Button(description='Paraphrase')
btn_run

Button(description='Paraphrase', style=ButtonStyle())

In [7]:
def on_click_classify(change):
    paraphrases = t5_paraphrase_sentence(text, num_gen)
    return paraphrases
    
btn_run.on_click(on_click_classify)